In [ ]:
import pandas as pd
# import dask.dataframe as dd  # Efficient handling of large data
import matplotlib.pyplot as plt
import seaborn as sns
import os

CSVs = [f"{year}.csv" for year in range(2000, 2005)]

for CSV in CSVs:
    # if not os.path.exists(CSV):
    #     print(f"{CSV} not found! Skipping...")
    #     continue  # Skip to the next file

    try:
        print(f"Reading {CSV}...")

        # Try reading with UTF-8 first
        data = pd.read_csv(CSV, encoding="utf-8")
    
    except UnicodeDecodeError:
        print(f"Error decoding {CSV} with UTF-8, trying Latin-1...")
        data = pd.read_csv(CSV, encoding="latin1")

    # Keep only required columns
    Columns_Needed = ["Year", "DayOfWeek", "TailNum", "ArrDelay","ArrTime","CRSArrTime" ,"DepDelay","DepTime" ,"CRSDepTime"]
    Updated_data = data[Columns_Needed]
    # Drop missing values
    Updated_data = Updated_data.dropna()

    # Create new filename
    NewCSV = CSV.replace(".csv", "_Q2AB.csv")
    
    # Save cleaned data
    Updated_data.to_csv(NewCSV, index=False)

    print(f"Processed: {CSV} → Saved as {NewCSV}")


years = range(2000, 2005)

# Dictionary to store DataFrames
data_dict = {}

# Read each CSV file
for year in years:
    file_name = f"{year}_Q2AB.csv"  
    # Check if file exists
    if os.path.exists(file_name): 
        print(f"Loading {file_name}...")
        data_dict[year] = pd.read_csv(file_name)
    else:
        print(f"File {file_name} not found!")

# Combine all DataFrames into one
if data_dict:
    combined_data = pd.concat(data_dict.values(), ignore_index=True)
    print("All files successfully loaded and combined!")
else:
    print("No files found.")


 # Create new CSV file for combined data from 2000 to 2004
combined_data.to_csv("Combined_Flight_Data_2000_2004.csv", index=False)
print("Combined data saved as 'Combined_Flight_Data_2000_2004.csv'")


df = pd.read_csv("Combined_Flight_Data_2000_2004.csv")
print(df)


df_Q2ab = df[['Year','CRSDepTime', 'DepDelay', 'ArrDelay','DayOfWeek']]
day_mapping = {
    1: "Monday",
    2: "Tuesday",
    3: "Wednesday",
    4: "Thursday",
    5: "Friday",
    6: "Saturday",
    7: "Sunday"
}
# Drop duplicates 

df_Q2ab=df_Q2ab.drop_duplicates()

# convert day of week from numbers to names
df_Q2ab["DayOfWeek"] = df_Q2ab["DayOfWeek"].map(day_mapping)
#Display only whole numbers
pd.options.display.float_format = '{:g}'.format
df_Q2ab['CRSDepTime'] = df_Q2ab['CRSDepTime'].apply(lambda x: '{0:04}'.format(int(x)))
df_Q2ab['CRSDepTime'] = pd.to_datetime(df_Q2ab['CRSDepTime'], format='%H%M', errors='coerce')
#Time_period['CRSDepTime'] = pd.to_datetime(Time_period['CRSDepTime'], errors='coerce')
df_Q2ab['CRSDepTime'] = df_Q2ab['CRSDepTime'].dt.strftime('%H:%M')

print(f"Number of duplicate rows: {df_Q2ab.duplicated().sum()}")
print(df_Q2ab.head(5))

#calculate total delay
df_Q2ab['TotalDelay'] = df_Q2ab['DepDelay'] + df_Q2ab['ArrDelay']
df_Q2ab = df_Q2ab[df_Q2ab['TotalDelay'] >= 0]
print(df_Q2ab)

# time group of 4 hour interval
def categorize_time_groups(depart_time, Time_Interval_minutes):  
    time_groups = pd.cut(                              
        pd.to_datetime(depart_time).dt.hour * 60 + pd.to_datetime(depart_time).dt.minute,
        bins=range(0, 1440, Time_Interval_minutes),  
        labels=False
    )
    return time_groups + 1  

Time_Interval_minutes = 239 

df_Q2ab['TimeGroup'] = categorize_time_groups(df_Q2ab['CRSDepTime'], Time_Interval_minutes)

df_Q2ab = df_Q2ab.dropna()

print(df_Q2ab)

#generate time interval of 4 hours
def generate_time_intervals(time_group):
    start_hour = int((time_group - 1) * 4)
    end_hour = start_hour + 3
    return f'{start_hour:02d}00hrs-{end_hour:02d}59hrs'

df_Q2ab['TimeInterval'] = df_Q2ab['TimeGroup'].apply(generate_time_intervals)

print(df_Q2ab.head(5))

# df_Q2ab = df_Q2ab.sort_values(by=['Year', 'TimeGroup'])

# Find mean using groupby in pandas
TODMean = df_Q2ab.groupby(['TimeInterval', 'Year'])['TotalDelay'].mean().reset_index()

# Rename columns
TODMean = TODMean.rename(columns={'TimeInterval': 'Time_Interval', 'Year': 'Year', 'TotalDelay': 'Mean'})
pd.options.display.float_format = '{:g}'.format
# TODMean['Mean'] = TODMean['Mean'].map(lambda x: f"{x:.0f}")
print(TODMean.dtypes)
print(TODMean.head(5)) 
print(df_Q2ab.head(5))